# Confirmatory Factor Analysis
CFA for testing JIT and environmental practice bundles

In [60]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from factor_analyzer import ConfirmatoryFactorAnalyzer, ModelSpecificationParser
from scipy.stats import norm

current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
PROCESSED_DATA_PATH = os.path.join(project_root, 'data', 'processed')
RAW_DATA_PATH = os.path.join(project_root, 'data', 'raw')
OUTPUT_PATH = os.path.join(project_root, 'output')

In [61]:
data = pd.read_excel(RAW_DATA_PATH + '/HPM data_environmental performance.xlsx')

In [62]:
environmental_practices = pd.DataFrame()
environmental_performance = pd.DataFrame()
jit_practices = pd.DataFrame()

for column in data.columns:
    if column.startswith('ENVRTX') or column.startswith('EPRACX'):
        environmental_practices[column] = data[column]

    if column.startswith('EPERFX'):
        environmental_performance[column] = data[column]

    if column.startswith('LAYOUT') or column.startswith('JITDEL') or column.startswith('KANBAN'):
        jit_practices[column] = data[column]

bundles = [jit_practices, environmental_practices, environmental_performance]

In [63]:
for bundle in bundles:
    print(bundle.shape)

(330, 10)
(330, 41)
(330, 9)


In [64]:
# drop rows with NA values
for bundle in bundles:
    bundle.dropna(inplace=True)

for bundle in bundles:
    print(bundle.shape)

(267, 10)
(243, 41)
(277, 9)


In [65]:
jit_practices

,LAYOUTN01,LAYOUTN02,LAYOUTN03,LAYOUTN04,JITDELN01,JITDELN02,JITDELN03,KANBANN01,KANBANN02,KANBANN03
3,3.0,3.0,2.0,3.0,1.0,1.0,2.0,1.0,3.0,3.0
6,5.0,3.0,5.0,4.0,4.0,2.0,1.0,3.0,1.0,3.0
7,3.0,2.0,4.0,2.0,2.0,3.5,3.5,1.0,1.0,4.0
8,4.0,4.0,4.0,4.0,3.0,4.0,2.0,1.0,2.0,2.0
9,4.0,4.0,4.0,4.0,2.0,5.0,2.0,1.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...
325,3.0,4.0,4.0,3.0,1.0,5.0,1.0,1.0,1.0,1.0
326,4.0,4.0,4.0,4.0,3.0,3.0,1.0,1.0,1.0,1.0
327,5.0,4.0,4.0,4.0,4.0,2.0,5.0,3.0,3.0,2.0
328,4.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,1.0,1.0


In [8]:
# # Model specification
# model_dict = {
#     "JIT Practices": []
# }

# model_dict = {
#     "JIT": [
#         "LAYOUTN02",
#         "KANBANN02",
#         "JITDELN02"
#     ]
# }

In [9]:
# for key in data.keys():
#     if key.startswith('LAYOUT') or key.startswith('JITDEL') or key.startswith('KANBAN'):
#         if key not in model_dict['JIT Practices']:
#             model_dict["JIT Practices"].append(key)

# model_dict["JIT Practices"].sort()

In [10]:
model_dict

NameError: name 'model_dict' is not defined

In [ ]:
# Extract all column names from the model_dict
desired_columns = [col for sublist in model_dict.values() for col in sublist]
data_filtered = data[desired_columns]
data_filtered.dropna(inplace=True)

# Adjusted model specification
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(data_filtered, model_dict)

# CFA model
cfa = ConfirmatoryFactorAnalyzer(model_spec)
cfa.fit(data_filtered)

# Extract the factor loadings
loadings = cfa.loadings_

# Get the standard errors for loadings and intercepts
se_all = cfa.get_standard_errors()

# Extract standard errors for loadings
se_loadings = se_all[0]

# Compute t-values
t_values = loadings / se_loadings

NameError: name 'model_dict' is not defined

In [91]:
data_filtered

,ENVRTX01,ENVRTX02,ENVRTX03,ENVRTX04,ENVRTX05,ENVRTX06,ENVRTX07,ENVRTX08,ENVRTX09,ENVRTX10,...,LAYOUTN04,EPERFX01,EPERFX02,EPERFX03,EPERFX04,EPERFX05,EPERFX06,EPERFX07,EPERFX08,EPERFX09
3,1.0,2.0,4.0,3.0,3.0,3.0,1.0,3.0,3.0,1.0,...,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0
8,3.0,3.0,4.0,4.0,3.0,3.0,2.0,3.0,2.0,2.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
9,3.0,4.0,5.0,4.0,5.0,5.0,4.0,5.0,4.0,3.0,...,4.0,5.0,5.0,3.0,4.0,5.0,4.0,3.0,5.0,1.0
11,1.0,4.0,3.0,3.0,4.0,4.0,1.0,3.0,5.0,1.0,...,1.0,4.0,3.0,3.0,5.0,4.0,5.0,4.0,4.0,5.0
12,2.0,3.0,5.0,4.0,4.0,3.0,1.0,5.0,5.0,2.0,...,4.0,5.0,4.0,4.0,4.0,1.0,1.0,3.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,4.0,3.0,4.0,4.0,4.0,4.0,1.0,3.0,1.0,2.0,...,2.0,4.0,3.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0
319,3.0,3.0,4.0,3.0,3.0,1.0,1.0,3.0,4.0,1.0,...,5.0,4.0,4.0,3.0,4.0,4.0,5.0,5.0,4.0,4.0
323,4.0,3.0,4.0,4.0,4.0,5.0,4.0,5.0,1.0,4.0,...,3.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,5.0
327,5.0,3.0,4.0,3.0,5.0,5.0,2.0,4.0,2.0,3.0,...,4.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [67]:
# Model specification
model_dict = {
    "Environmental Practices": [],
    "JIT Practices": [],
    "Environmental Performance": []
}

In [76]:
for key in data.keys():
    if key.startswith('ENVRTX') or key.startswith('EPRACX'):
        if key not in model_dict['Environmental Practices']:
            model_dict['Environmental Practices'].append(key)

    if key.startswith('LAYOUT') or key.startswith('JITDEL') or key.startswith('KANBAN'):
        if key not in model_dict['JIT Practices']:
            model_dict["JIT Practices"].append(key)

    if key.startswith('EPERFX'):
        if key not in model_dict['Environmental Performance']:
            model_dict["Environmental Performance"].append(key)

model_dict["Environmental Practices"].sort()
model_dict["JIT Practices"].sort()
model_dict["Environmental Performance"].sort()

In [77]:
model_dict

{'Environmental Practices': ['ENVRTX01',
  'ENVRTX02',
  'ENVRTX03',
  'ENVRTX04',
  'ENVRTX05',
  'ENVRTX06',
  'ENVRTX07',
  'ENVRTX08',
  'ENVRTX09',
  'ENVRTX10',
  'ENVRTX11',
  'ENVRTX12',
  'ENVRTX13',
  'ENVRTX14',
  'ENVRTX15',
  'ENVRTX17',
  'ENVRTX18',
  'ENVRTX20',
  'ENVRTX21',
  'ENVRTX22',
  'ENVRTX23',
  'ENVRTX24',
  'ENVRTX29',
  'ENVRTX30',
  'ENVRTX31',
  'ENVRTX32',
  'ENVRTX33',
  'ENVRTX34',
  'ENVRTX35',
  'ENVRTX36',
  'ENVRTX37',
  'ENVRTX38',
  'ENVRTX39',
  'ENVRTX40',
  'ENVRTX41',
  'EPRACX01',
  'EPRACX02',
  'EPRACX03',
  'EPRACX04',
  'EPRACX05',
  'EPRACX06'],
 'JIT Practices': ['JITDELN01',
  'JITDELN02',
  'JITDELN03',
  'KANBANN01',
  'KANBANN02',
  'KANBANN03',
  'LAYOUTN01',
  'LAYOUTN02',
  'LAYOUTN03',
  'LAYOUTN04'],
 'Environmental Performance': ['EPERFX01',
  'EPERFX02',
  'EPERFX03',
  'EPERFX04',
  'EPERFX05',
  'EPERFX06',
  'EPERFX07',
  'EPERFX08',
  'EPERFX09']}

In [82]:
# Extract all column names from the model_dict
desired_columns = [col for sublist in model_dict.values() for col in sublist]
data_filtered = data[desired_columns]
data_filtered.dropna(inplace=True)

/var/folders/l3/xnhphxjs6_x60z1z798fy0dw0000gq/T/ipykernel_97837/1215649990.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered.dropna(inplace=True)


In [92]:
eprx_median = np.median(environmental_practices)
eprf_median = np.median(environmental_performance)
jit_median = np.median(jit_practices)
print("Environmental Practices median: ", eprx_median)
print("Environmental Performance median: ", eprf_median)
print("JIT Practices median: ", jit_median)

Environmental Practices median:  3.5
Environmental Performance median:  4.0
JIT Practices median:  3.5


In [102]:
high_environmental_practices = environmental_practices[environmental_practices >= eprx_median]
low_environmental_practices = environmental_practices[environmental_practices < eprx_median]

In [103]:
overall_median = environmental_performance.stack().median()

TypeError: could not convert string to float: 'Above Median'

In [ ]:
row_medians = environmental_performance.median(axis=1)

In [104]:
row_medians

0      5.0
2      4.0
3      3.0
4      3.0
5      3.0
      ... 
324    5.0
325    5.0
327    5.0
328    3.0
329    3.5
Length: 277, dtype: float64

In [109]:
environmental_performance['Performance'] = np.where(row_medians > overall_median, "High", "Low")

In [110]:
environmental_performance

,EPERFX01,EPERFX02,EPERFX03,EPERFX04,EPERFX05,EPERFX06,EPERFX07,EPERFX08,EPERFX09,Row Status,Performance Status,Performance
0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,Above Median,High,High
2,4.0,4.0,4.0,4.0,1.0,2.0,4.0,4.0,1.0,Below Median,Low,Low
3,3.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,Below Median,Low,Low
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,Below Median,Low,Low
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,Below Median,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...
324,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,Above Median,High,High
325,5.0,3.0,4.0,4.0,5.0,5.0,4.0,5.0,5.0,Above Median,High,High
327,5.0,4.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,Above Median,High,High
328,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,Below Median,Low,Low


In [113]:
# split high and low performance
high_performance = environmental_performance[environmental_performance['Performance'] == "High"]
low_performance = environmental_performance[environmental_performance['Performance'] == "Low"]

In [121]:
import pandas as pd
from scipy import stats

# Assuming you have already created the 'high_performance' and 'low_performance' DataFrames

# Columns to compare
columns_to_compare = ['EPERFX01', 'EPERFX02', 'EPERFX03', 'EPERFX04', 'EPERFX05', 'EPERFX06', 'EPERFX07', 'EPERFX08', 'EPERFX09']

# Loop through each column and perform a t-test
for column in columns_to_compare:
    high_group = high_performance[column].dropna()
    low_group = low_performance[column].dropna()

    # Perform t-test
    t_stat, p_value = stats.ttest_ind(high_group, low_group)

    # Output the results
    print(f"Column: {column}")
    print(f"  T-Statistic: {t_stat}")
    print(f"  P-Value: {p_value}")

    # Determine if the difference is statistically significant
    alpha = 0.05  # You can adjust the significance level as needed
    if p_value < alpha:
        print("  Statistically significant difference detected")
    else:
        print("  No statistically significant difference detected")



Column: EPERFX01
  T-Statistic: 10.070063638558121
  P-Value: 1.651826758734626e-20
  Statistically significant difference detected
Column: EPERFX02
  T-Statistic: 9.31600799068227
  P-Value: 4.053588587564886e-18
  Statistically significant difference detected
Column: EPERFX03
  T-Statistic: 11.064002405843752
  P-Value: 8.845489421968754e-24
  Statistically significant difference detected
Column: EPERFX04
  T-Statistic: 11.152345501848924
  P-Value: 4.468953915111016e-24
  Statistically significant difference detected
Column: EPERFX05
  T-Statistic: 10.212403594700483
  P-Value: 5.719720118401109e-21
  Statistically significant difference detected
Column: EPERFX06
  T-Statistic: 9.289009339833418
  P-Value: 4.917430871664513e-18
  Statistically significant difference detected
Column: EPERFX07
  T-Statistic: 10.16832672434529
  P-Value: 7.948809241231352e-21
  Statistically significant difference detected
Column: EPERFX08
  T-Statistic: 9.61021054949646
  P-Value: 4.85154068006694e-19

In [1]:
# Calculate the row-wise mean for the 'Low' performance group
low_performance['Low_Mean'] = low_performance[['EPERFX01', 'EPERFX02', 'EPERFX03', 'EPERFX04', 'EPERFX05', 'EPERFX06', 'EPERFX07', 'EPERFX08', 'EPERFX09']].mean(axis=1)

# Calculate the row-wise mean for the 'High' performance group
high_performance['High_Mean'] = high_performance[['EPERFX01', 'EPERFX02', 'EPERFX03', 'EPERFX04', 'EPERFX05', 'EPERFX06', 'EPERFX07', 'EPERFX08', 'EPERFX09']].mean(axis=1)

# Perform a t-test to compare the row-wise means between 'Low' and 'High'
t_stat, p_value = stats.ttest_ind(low_performance['Low_Mean'].dropna(), high_performance['High_Mean'].dropna())

# Output the results
print("Comparison of row-wise means between 'Low' and 'High' performance groups")
print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_value}")

# Determine if the difference is statistically significant
alpha = 0.05  # You can adjust the significance level as needed
if p_value < alpha:
    print("Statistically significant difference detected")
else:
    print("No statistically significant difference detected")

NameError: name 'low_performance' is not defined

## Constructing Dummy Variables

,ENVRTX01,ENVRTX02,ENVRTX03,ENVRTX04,ENVRTX05,ENVRTX06,ENVRTX07,ENVRTX08,ENVRTX09,ENVRTX10,...,LAYOUTN04,EPERFX01,EPERFX02,EPERFX03,EPERFX04,EPERFX05,EPERFX06,EPERFX07,EPERFX08,EPERFX09
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,0,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,0,1,0
11,0,1,0,0,1,1,0,0,1,0,...,0,1,0,0,1,1,1,1,1,1
12,0,0,1,1,1,0,0,1,1,0,...,1,1,1,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,1,0,1,1,1,1,0,0,0,0,...,0,1,0,1,1,0,0,1,1,1
319,0,0,1,0,0,0,0,0,1,0,...,1,1,1,0,1,1,1,1,1,1
323,1,0,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,1,1
327,1,0,1,0,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,1,1


In [20]:
data = {
    'Bundle': [],
    'Item description': [],
    'Loading': [],
    'SE': [],
    't-value': []
}

# Loop through each factor and then each item within that factor
for factor, items in model_dict.items():
    for item in items:
        item_idx = data_filtered.columns.get_loc(item)
        factor_idx = list(model_dict.keys()).index(factor)

        loading_value = loadings[item_idx][factor_idx]
        se_value = se_loadings[item_idx][factor_idx]
        t_value = t_values[item_idx][factor_idx]

        data['Bundle'].append(factor)
        data['Item description'].append(item)  # Using column names as descriptions for now
        data['Loading'].append(loading_value)
        data['SE'].append(se_value)
        data['t-value'].append(t_value)

# Convert the data to a DataFrame
results_df = pd.DataFrame(data)
print(results_df)


                       Bundle Item description   Loading        SE    t-value
0     Environmental Practices         ENVRTX01  0.936079  0.069555  13.458204
1     Environmental Practices         ENVRTX02  0.964870  0.069595  13.864165
2     Environmental Practices         ENVRTX03  0.895810  0.063026  14.213274
3     Environmental Practices         ENVRTX04  0.882754  0.062851  14.045141
4     Environmental Practices         ENVRTX05  0.926119  0.064761  14.300651
5     Environmental Practices         ENVRTX06  0.934617  0.069959  13.359559
6     Environmental Practices         ENVRTX07  1.065586  0.081706  13.041634
7     Environmental Practices         ENVRTX08  0.910765  0.064602  14.098101
8     Environmental Practices         ENVRTX09  1.041666  0.076772  13.568255
9     Environmental Practices         ENVRTX10  1.028412  0.072222  14.239538
10    Environmental Practices         ENVRTX11  1.036353  0.073577  14.085292
11    Environmental Practices         ENVRTX12  0.964537  0.0800

In [21]:
# Compute p-values from t-values
p_values = [2 * (1 - norm.cdf(abs(t))) for t in data['t-value']]  # Two-tailed test

# Add p-values to the results dataframe
results_df['p-value'] = p_values

results_df

,Bundle,Item description,Loading,SE,t-value,p-value
0,Environmental Practices,ENVRTX01,0.936079,0.069555,13.458204,0.0
1,Environmental Practices,ENVRTX02,0.964870,0.069595,13.864165,0.0
2,Environmental Practices,ENVRTX03,0.895810,0.063026,14.213274,0.0
3,Environmental Practices,ENVRTX04,0.882754,0.062851,14.045141,0.0
4,Environmental Practices,ENVRTX05,0.926119,0.064761,14.300651,0.0
5,Environmental Practices,ENVRTX06,0.934617,0.069959,13.359559,0.0
6,Environmental Practices,ENVRTX07,1.065586,0.081706,13.041634,0.0
7,Environmental Practices,ENVRTX08,0.910765,0.064602,14.098101,0.0
8,Environmental Practices,ENVRTX09,1.041666,0.076772,13.568255,0.0
9,Environmental Practices,ENVRTX10,1.028412,0.072222,14.239538,0.0


In [22]:
import numpy as np
from scipy.stats import norm

# Assuming 'data' is a pandas DataFrame and 't-value' is a column in that DataFrame
formatted_p_values = []
significance_levels = []

for t in results_df['t-value']:
    p_value = 2 * (1 - norm.cdf(abs(t)))
    if p_value < 0.001:
        formatted_p_values.append(f"{p_value:.2e}")
        significance_levels.append('***')
    elif p_value < 0.01:
        formatted_p_values.append(f"{p_value:.2e}")
        significance_levels.append('**')
    elif p_value < 0.05:
        formatted_p_values.append(f"{p_value:.2e}")
        significance_levels.append('*')
    else:
        formatted_p_values.append(f"{p_value:.2e}")
        significance_levels.append('')

# Add the formatted p-values and significance levels as new columns to the DataFrame
results_df['formatted_p_value'] = formatted_p_values
results_df['significance'] = significance_levels

# Display the DataFrame
print(results_df[['t-value', 'formatted_p_value', 'significance']])


      t-value formatted_p_value significance
0   13.458204          0.00e+00          ***
1   13.864165          0.00e+00          ***
2   14.213274          0.00e+00          ***
3   14.045141          0.00e+00          ***
4   14.300651          0.00e+00          ***
5   13.359559          0.00e+00          ***
6   13.041634          0.00e+00          ***
7   14.098101          0.00e+00          ***
8   13.568255          0.00e+00          ***
9   14.239538          0.00e+00          ***
10  14.085292          0.00e+00          ***
11  12.042018          0.00e+00          ***
12  13.108859          0.00e+00          ***
13  14.529116          0.00e+00          ***
14  14.333659          0.00e+00          ***
15  13.405697          0.00e+00          ***
16  14.145885          0.00e+00          ***
17  14.234457          0.00e+00          ***
18  13.556738          0.00e+00          ***
19  13.505839          0.00e+00          ***
20  13.608903          0.00e+00          ***
21  15.304

In [23]:
# To save the results to a CSV file
results_df.to_csv(OUTPUT_PATH + '/cfa_explore_final.csv', index=False)